In [ ]:
import numpy as np
import pandas as pd
pd.set_option('max_rows', None, 'max_columns', None, 'expand_frame_repr', False)

In [ ]:
airports1 = pd.read_csv('us_airports.csv')
airports1

In [ ]:
airports2 = pd.read_csv('summary.csv')
airports2

In [ ]:
raw_airports = pd.merge(airports1, airports2, left_on='iata_code', right_on='Origin', how='inner')
raw_airports

In [ ]:
airports = raw_airports[['iata_code', 'type', 'name', 'lat', 'lon', 'elevation_ft', 'iso_region', 'municipality', 
                         'icao_code', 'Count', 'DepDelay', '15minDepDelay', 'Count2', 'ArrDelay', '15minArrDelay']]
airports = airports.dropna()
airports = airports.rename(columns={'name': 'fullname'})
airports

In [ ]:
def combine(x):
    return x['iso_region'][3:] + '.' + x['iata_code']

In [ ]:
airports['name'] = airports.apply(combine, axis=1)
airports

In [ ]:
airportsDict = airports[['iata_code', 'name']].set_index('iata_code').to_dict()['name']
airportsDict

In [ ]:
def convertName(x):
    return airportsDict[x]

In [ ]:
connection = pd.read_csv('airportconnection.csv')
connection = connection[connection['Origin'].isin(airports['iata_code'])]
connection = connection[connection['Dest'].isin(airports['iata_code'])]
connection

In [ ]:
connection['Origin'] = connection['Origin'].map(lambda x : airportsDict[x])
connection['Dest'] = connection['Dest'].map(airportsDict)
connection

In [ ]:
destcount = connection.groupby('Origin', as_index=False).count()
destcount

In [ ]:
newairports = pd.merge(destcount, airports, left_on='Origin', right_on='name', how='inner')
newairports

In [ ]:
dests = connection.groupby('Origin')['Dest'].apply(list).to_frame()
dests['Origin'] = dests.index
dests

In [ ]:
result = pd.merge(dests, newairports, on='Origin', how='inner')
result

In [ ]:
result = result.rename(columns={'Dest_x': 'imports', 'Dest_y': 'size'})
result = result.drop('Origin', axis=1)
result

In [ ]:
output = result.to_dict('records')
output

In [ ]:
import json
with open('hierarchy.json', 'w') as f:
    json.dump(output, f)